# Log文件解析脚本
 
目前，已经可以从Log中解析以下表格中的内容：  

|类型|正则名称|数据名称|  
|:-|:-|:-|
|MCLoc  |mcloc_re|t, x, y, theta, confidence|  
|Yaw|yaw_re|yaw_t, yaw_data, yaw_time, ax, ay, az, gx, gy, gz, offx, offy, offz|  
|Odometer|odometer_re|odo_t, odo_time, odo_x, odo_y, odo_theta|  
|手控下发速度|vel_re|vx, vy, omega|  
|自动下发速度|send_vel_re|send_vel_t, send_vx, send_vy, send_vw, send_steer_angle|  
|自动获取速度|get_vel_re|get_vel_t, get_vel_x, get_vy, get_vel_vw, get_vel_steer_angle|  

##  如何添加新的解析内容
1. [正则模块中添加新的正则表达式和对应的变量](#No1)  
2. [解析函数中处理解析出来的数据，赋予对应变量](#No2)  
3. [运行解析函数，并且画图](#No3)

In [1]:
%matplotlib qt5
import matplotlib.pyplot as plt
import os
import re
import numpy as np
import math
from datetime import datetime

## <span id="No1">正则 </span>

* 定位

In [2]:
mcloc_re = re.compile("\[(.*?)\]\[MCLocV2\].*x:(.*?):y:(.*?):theta:(.*?):confidence:(.*?) ")
t = []
x = []
y = []
theta =[]
confidence = []

 * 陀螺仪

In [3]:
yaw_re = re.compile("\[(.*?)\].*yaw: (.*?),? ?timestamp: (\d+)")
yaw_t = []
yaw_data = []
yaw_time = []
acc_re = re.compile("acc = \[(.*?), (.*?), (.*?)\]w = \[(.*?), (.*?), (.*?)\];off = \[(.*?), (.*?), (.*?)\]")
ax = []
ay = []
az = []
gx = []
gy = []
gz = []
offx = []
offy = []
offz = []

* 里程

In [4]:
odometer_re = re.compile("\[(.*?)\].*timestamp: (\d+), x: (.*?), y: (.*?), angle: (.*?), stopped:*?")
odo_t = []
odo_time = []
odo_x = []
odo_y = []
odo_theta = []
#odo_vx = []
#odo_vy = []
#odo_vw = []

* 下发速度

In [5]:
vel_re = re.compile('\[(.*?)\].*{ "vx" : (.*?), "vy" : (.*?), "w" : (.*?) }')
vel_t = []
vx =[]
vy = []
omega = []

* 自动下发速度

In [6]:
send_vel_re = re.compile('\[(.*?)\].* SEND vx = (.*?); vw = (.*?); steer_angle = (.*?); max_vx = (.*?); max_vw = (.*?)\n')
send_vel_t = []
send_vx = []
send_vw = []
send_steer= []
send_max_vx = []
send_max_vw = []

* 自动获取速度

In [8]:
get_vel_re = re.compile('\[(.*?)\].* GET vx = (.*?); vw = (.*?); steer_angle = (.*?); max_vx = (.*?); max_vw = (.*?)\n')
get_vel_t = []
get_vx = []
get_vw = []
get_steer = []
get_max_vx = []
get_max_vw = []

* LQR的参数Q

In [7]:
get_Q_re = re.compile('\[(.*?)\].* m_Q.*? = (.*?);.*\n')
get_Q_t = []
get_Q_q = []

## <span id="No2">解析函数 </span>

In [9]:
def rbktimetodate(rbktime):
    return datetime.strptime(rbktime, '%Y-%m-%d %H:%M:%S.%f')
def findrange(ts, t1, t2):
    small_ind = -1
    large_ind = len(ts)-1
    for i, data in enumerate(ts):
        large_ind = i
        if(t1 < data and small_ind < 0):
            small_ind = i
        if(t2 < data):
            break;
    return small_ind, large_ind
def anglejoint(anglelist, period = 360.0):
    base_offset = anglelist[0]
    anglelist[0] = 0
    noise = 0.01 * 360
    print(base_offset)
    for i in range(1, len(anglelist)):
        temp = anglelist[i] - base_offset
        if(temp - anglelist[i - 1] > (period-noise)):
            anglelist[i] =temp - period
        elif (temp - anglelist[i - 1] < (-period+noise)):
            anglelist[i] =temp + period
        else:
            anglelist[i] = temp
        if(anglelist[i] > period/2.0):
            anglelist[i] -= period
        elif(anglelist[i] < -period/2.0):
            anglelist[i] += period
def read_log(filename): 
    line_num = 0
    for line in open(filename): 
        line_num += 1
        #print(line_num)
        m = mcloc_re.match(line)
        yaw = yaw_re.match(line)
        odo = odometer_re.match(line)
        acc = acc_re.match(line)
        vel = vel_re.match(line)
        send_vel = send_vel_re.match(line)
        get_vel = get_vel_re.match(line)
        get_Q = get_Q_re.match(line)
        if m:
            t.append(rbktimetodate(m.group(1)))
            x.append(float(m.group(2)))
            y.append(float(m.group(3)))
            theta.append(float(m.group(4)))
            confidence.append(float(m.group(5)))
        if yaw:
            yaw_t.append(rbktimetodate(yaw.group(1)))
            yaw_data.append((float(yaw.group(2))/math.pi *  180.0))
            yaw_time.append(float(yaw.group(3)))
        if acc:
            ax.append(float(acc.group(1)))
            ay.append(float(acc.group(2)))
            az.append(float(acc.group(3)))
            gx.append(float(acc.group(4))/math.pi*180.0*16.4)
            gy.append(float(acc.group(5))/math.pi*180.0*16.4)
            gz.append(float(acc.group(6))/math.pi*180.0*16.4)
            offx.append(float(acc.group(7)))
            offy.append(float(acc.group(8)))
            offz.append(float(acc.group(9)))
        if odo:
            odo_t.append(rbktimetodate(odo.group(1)))
            odo_time.append(float(odo.group(2)))
            odo_x.append(float(odo.group(3)))
            odo_y.append(float(odo.group(4)))
            odo_theta.append((float(odo.group(5))/math.pi * 180.0))
            #odo_vx.append((float(odo.group(6))))
            #odo_vy.append((float(odo.group(7))))
            #odo_vw.append(float(odo.group(8)))
        if vel:
            vel_t.append(rbktimetodate(vel.group(1)))
            vx.append(float(vel.group(2)))
            vy.append(float(vel.group(3)))
            omega.append(float(vel.group(4)))
        if send_vel:
            send_vel_t.append(rbktimetodate(send_vel.group(1)))
            send_vx.append(float(send_vel.group(2)))
            send_vw.append(float(send_vel.group(3)))
            send_steer.append(float(send_vel.group(4)))
            send_max_vx.append(float(send_vel.group(5)))
            send_max_vw.append(float(send_vel.group(6)))
        if get_vel:
            get_vel_t.append(rbktimetodate(get_vel.group(1)))
            get_vx.append(float(get_vel.group(2)))
            get_vw.append(float(get_vel.group(3)))
            get_steer.append(float(get_vel.group(4)))
            get_max_vx.append(float(get_vel.group(5)))
            get_max_vw.append(float(get_vel.group(6)))
        if get_Q:
            get_Q_t.append(rbktimetodate(get_Q.group(1)))
            get_Q_q.append(float(get_Q.group(2)))

## <span id="No3">解析文件，并画图 </span>

In [10]:
filename= 'robokit_2018-07-12_11-42-21.695.log'
#filename = 'F:\\dazu\\robokit_2012-01-11_13-44-13.827\\robokit_2012-01-11_13-44-13.827.log'
t, x, y, theta, confidence = [], [], [], [], []
yaw_t, yaw_data, yaw_time = [], [], []
ax, ay, az = [], [], []
gx, gy, gz = [], [], []
offx, offy, offz = [], [], []
odo_t, odo_time, odo_x, odo_y, odo_theta, odo_vx, odo_vy, odo_vw = [], [], [], [], [], [], [], []
vel_t, vx, vy, omega = [], [], [], []
send_vel_t, send_vx, send_vw, send_steer, send_max_vx, send_max_vw = [], [], [], [], [], []
get_vel_t, get_vx, get_vw, get_steer, get_max_vx, get_max_vw = [], [], [], [], [], []
get_Q_t, get_Q_q = [],[]
read_log(filename)
#anglejoint(theta)
#anglejoint(yaw_data)
#anglejoint(odo_theta)

* 下发速度

In [16]:
plt.figure(11)
#plt.plot(send_vel_t,send_max_vx, 'o',label='max vx')
plt.plot(send_vel_t,send_vx, '.',label='send vel')
plt.plot(get_vel_t,get_vx, '.',label='get')
plt.ylabel('vx (m/s)')
plt.xlabel('time (s)')
plt.legend(loc = 'lower right')
plt.grid()
plt.figure(12)
#plt.plot(send_vel_t,send_max_vy, 'o',label='max vy')
plt.plot(send_vel_t,send_vw, 'o',label='send vy')
plt.plot(get_vel_t,get_vw, '.',label='get')
plt.ylabel('vw (rad/s)')
plt.xlabel('time (s)')
plt.legend(loc = 'lower right')
plt.grid()
plt.figure(13)
#plt.plot(send_vel_t,send_max_vy, 'o',label='max vy')
plt.plot(send_vel_t,send_steer, 'o',label='send vy')
plt.plot(get_vel_t,get_steer, '.',label='get')
plt.ylabel('steer_angle (rad)')
plt.xlabel('time (s)')
plt.legend(loc = 'lower right')
plt.grid()
plt.figure(14)
plt.plot(get_Q_t,get_Q_q,'.',label= 'get_Q')
plt.legend()

* 轨迹

In [13]:
plt.figure(3)
plt.plot(t,x,label='x')
#plt.plot(odo_t,[tmp - odo_x[0] +x[0] for tmp in odo_x],label='odo_x')
plt.xlabel('t')
plt.ylabel('x (m)')
plt.grid()
plt.legend()
plt.figure(4)
plt.plot(t,y,label='y')
#plt.plot(odo_t,[tmp - odo_y[0] +y[0] for tmp in odo_y],label='odo_y')
plt.legend()
plt.figure(5)
plt.plot(x,y,'k.')
plt.figure(6)
plt.plot(t,theta)
plt.xlabel('t')
plt.ylabel('theta')
plt.figure(7)
plt.plot(t,confidence)
plt.xlabel('t')
plt.ylabel('confidence')

Text(0,0.5,'confidence')

* 角度

In [20]:
plt.figure(122)
plt.plot(t,theta,'g--',label='mcloc ')
plt.plot(odo_t,[tmp + theta[0] - odo_theta[0] for tmp in odo_theta],'r--',label='Odo ')
plt.plot(yaw_t,[tmp + theta[0] - yaw_datafor tmp in yaw_data],'k--',label='Gyro ')
plt.legend()

* 地图线路和实际线路比较

In [48]:
import bezier
import numpy as np
from scipy import interpolate
nodes = np.asfortranarray([[-1.586, -0.977, 1.173, 4.936],[1.77, 0.441 , 0.175, 0.181]])
curve = bezier.Curve(nodes, degree=3)
ax = curve.plot(num_pts=256)
ax.plot(x,y,'k.')
ax.legend(['Path in Map','Real Trajectory'])
plt.gcf()
plt.xlabel('x(m)')
plt.ylabel('y(m)')

* 一段时间内的数据

In [129]:
t1 = rbktimetodate("2018-02-10 14:29:07.0")
t2 = rbktimetodate("2018-02-10 14:29:26.0")
ind1, ind2 = findrange(t, t1, t2)
plt.figure(2)
plt.subplot(3,1,1)
plt.title('Go Strght Directly')
plt.plot(t[ind1:ind2],theta[ind1:ind2],'g--',label='mcloc ')
plt.legend(loc = 'upper right')
ind1, ind2 = findrange(odo_t, t1, t2)
plt.subplot(3,1,2)
plt.plot(odo_t[ind1:ind2],odo_theta[ind1:ind2],'r--',label='Odo ')
plt.legend(loc = 'upper right')
ind1, ind2 = findrange(yaw_t, t1, t2)
plt.subplot(3,1,3)
plt.plot(yaw_t[ind1:ind2],yaw_data[ind1:ind2],'k--',label='Gyro ')
plt.legend(loc = 'upper right')